# 导入包和数据

In [637]:
import pandas
import numpy
import gurobipy as gp
import json

In [638]:
# read data
with open("sorted_ToyData_dict.json") as json_file:
     toy_data_dict= json.load(json_file)
with open("location.json") as json_file:
     location = json.load(json_file)
with open("slot.json") as json_file:
     slot = json.load(json_file)
with open("data_req.json") as json_file:
     demand = json.load(json_file)
with open("bandwidth.json") as json_file:
     bandwidth = json.load(json_file)

# 初始化常量

In [639]:
# number of jobs and slots
K = len(toy_data_dict)
J = len(slot)

In [640]:
# number of tasks in a job in stage 1
L = []
for key in toy_data_dict.keys():
    L.append(len(toy_data_dict[key]['stage']['1']))

In [641]:
# slots number in a data center
a = [];
for val in slot.values():
    a.append(val)

In [642]:
# execution time of certain tasks
e = []
for i in toy_data_dict.keys():
    exc = []
    for j in toy_data_dict[i]['Execution Time'].values():
        exc.append(j)
    e.append(exc)

In [643]:
# all tasks in stage 1
tasks_1 = []
for i in toy_data_dict.keys():
    tasks_1.append(toy_data_dict[i]['stage']['1'])

In [645]:
# normalize location
for (key,val) in location.items():
    s = val[2:]
    location[key] = int(s)

In [646]:
# communication time matrix
c = []
for k in range(K):
    com = []
    for i in range(len(tasks_1[k])):
        compute = []
        for j in range(J):
            maximum = 0
            for loc in demand[tasks_1[k][i]]:
                src = location[loc] - 1
                time = demand[tasks_1[k][i]][loc]/bandwidth[src][j]
                if(time > maximum):
                    maximum = time
            compute.append(maximum)
        com.append(compute)
    c.append(com)

In [647]:
# completion time records
complete_time = {}
job_complete_time = {}

In [648]:
# completed job records
complete_jobs = []

In [649]:
# record global time
global_time = 0

In [650]:
# record current stages and total stages
stages = [1,1,1,1,1,1]
total_stages = []
s = ''
for i in toy_data_dict.keys():
    for j in toy_data_dict[i]['stage'].keys():
        s = int(j)
    total_stages.append(s)

In [651]:
# slots occupied by jobs
occ_slots = {}

# 建立线性模型

In [185]:
# Create a ILP model
m = gp.Model()

In [186]:
x = []
X = []
for k in range(K):
    x.append(0)
    X.append(0)

In [187]:
for k in range(K):
    x[k] = m.addVars(L[k],J,2,name="lambda"+str(k))
    X[k] = m.addVars(L[k],J,name="x"+str(k))

In [188]:
m.update()

In [189]:
m.setObjective(
    gp.quicksum(x[k][i,j,0]+x[k][i,j,1]*pow(M,c[k][i][j]+e[k][i])
               for k in range(K) for i in range(L[k]) for j in range(J)),
    gp.GRB.MINIMIZE
)

In [190]:
m.addConstrs(X[k][i,j] == x[k][i,j,1] 
             for k in range(K) for i in range(L[k]) for j in range(J))
m.addConstrs(x[k][i,j,0] + x[k][i,j,1] == 1
             for k in range(K) for i in range(L[k]) for j in range(J))
m.addConstrs(gp.quicksum(X[k][i,j] for k in range(K) for i in range(L[k]))
            <= a[j] for j in range(J))
m.addConstrs(gp.quicksum(X[k][i,j] for j in range(J)) == 1
            for k in range(K) for i in range(L[k]))

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (4, 0): <gurobi.Constr *Awaiting Model Update*>,
 (5, 0): <gurobi.Constr *Awaiting Model Update*>}

In [191]:
m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 229 rows, 312 columns and 624 nonzeros
Model fingerprint: 0xfdfcecbd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+14]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 208 rows and 208 columns
Presolve time: 0.01s
Presolved: 21 rows, 104 columns, 208 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0400000e+02   8.000000e+00   0.000000e+00      0s
       8    1.7322726e+09   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds
Optimal objective  1.732272583e+09


# 第一轮迭代结果处理

In [193]:
res = []

In [194]:
for var in m.getVars():
    if(var.X == 1.0 and var.varName[0] == 'x'):
        res.append(int(var.varName[5:-1])+1)

In [196]:
tasks_1_unfold = []
for i in range(K):
    for j in range(len(tasks_1[i])):
        tasks_1_unfold.append(tasks_1[i][j])

In [197]:
num = 0
maxim = 0
bottleneck = 0
for i in range(K):
    for j in range(len(tasks_1[i])):
        time = c[i][j][res[num]-1]
        if(time > maxim):
            maxim = time
            bottleneck = i
        num += 1

In [199]:
setted = []
num = 0
for i in range(K):
    for j in range(len(tasks_1[i])):
        if(i == bottleneck):
            setted.append(res[num]-1)
        num += 1

# 线性求解接口实现

In [652]:
# 定义线性函数接口
def LP(K, L, avai, M):
    m = gp.Model()
    x = [0] * w
    X = [0] * w
    for k in K:
        x[k] = m.addVars(L[k],J,2,name="lambda"+str(k))
        X[k] = m.addVars(L[k],J,name="x"+str(k))
    m.update()
    m.setObjective(
    gp.quicksum(x[k][i,j,0]+x[k][i,j,1]*pow(M,c[k][i][j]+e[k][i])
            for k in K for i in range(L[k]) for j in range(J)),
    gp.GRB.MINIMIZE
    )
    m.addConstrs(X[k][i,j] == x[k][i,j,1] 
            for k in K for i in range(L[k]) for j in range(J))
    m.addConstrs(x[k][i,j,0] + x[k][i,j,1] == 1
            for k in K for i in range(L[k]) for j in range(J))
    m.addConstrs(gp.quicksum(X[k][i,j] for k in K for i in range(L[k]))
            <= avai[j] for j in range(J))
    m.addConstrs(gp.quicksum(X[k][i,j] for j in range(J)) == 1
            for k in K for i in range(L[k]))
    m.optimize()
    print(m.objVal)
    res = []
    for var in m.getVars():
        if(var.X == 1.0 and var.varName[0] == 'x'):
            res.append(int(var.varName[5:-1])+1)
    return res

In [653]:
def update(K, avai, M):
    num = 0
    maxim = 0
    bottleneck = 0
    for i in K:
        for j in range(len(tasks_new[i])):
            time = c[i][j][res[num]-1] + e[i][j]
            if(time > maxim):
                maxim = time
                bottleneck = i
            num += 1
    complete_time[bottleneck] = global_time + maxim
    occ_slots[bottleneck] = []
    setted = []
    num = 0
    for i in K:
        for j in range(len(tasks_new[i])):
            if(i == bottleneck):
                setted.append(res[num]-1)
                occ_slots[bottleneck].append(res[num]-1)
                location[tasks_new[i][j]] = res[num]
            num += 1
    K.remove(bottleneck)
    M = 0
    for i in K:
        M += J * L[i]
    for i in setted:
        avai[i] = avai[i] - 1
    return K, avai, M

# 完全调用接口重新实现第一轮迭代

In [654]:
K = [0,1,2,3,4,5]
# 注意copy()!
avai = a.copy()
M = 0
w = 6
tasks_new = tasks_1.copy()
for i in K:
        M += J * L[i]
for i in range(w):
    res = LP(K,L,avai,M)
    K, avai, M = update(K, avai, M)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 229 rows, 312 columns and 624 nonzeros
Model fingerprint: 0xfdfcecbd
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 6e+14]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 208 rows and 208 columns
Presolve time: 0.01s
Presolved: 21 rows, 104 columns, 208 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0400000e+02   8.000000e+00   0.000000e+00      0s
       8    1.7322726e+09   0.000000e+00   0.000000e+00      0s

Solved in 8 iterations and 0.02 seconds
Optimal objective  1.732272583e+09
1732272583.4946697
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, usin

# 第二轮stages调度

In [655]:
# set thresholds for rescheduling
time_threshold = 1
job_threshold = 2

In [190]:
# 分析在阈值之下完成的jobs
num = 0
wait = 0
start = complete_time_order[0][1]
complete_jobs.clear()
for i in range(job_threshold):
    wait = (complete_time_order[i][1] - start)
    if(wait > time_threshold):
        global_time = start + time_threshold
        break
    global_time = complete_time_order[i][1]
    complete_jobs.append(complete_time_order[i][0])

In [191]:
# 更新中间结果信息
for i in complete_jobs:
    if(stages[i] == total_stages[i]):
        complete_jobs.remove(i)
        job_complete_time[i] = complete_time_order[i][1]
        del complete_time[i]
    for j in occ_slots[i]:
        avai[j] += 1
    occ_slots[i].clear()    

In [192]:
tasks_new = []
K = complete_jobs.copy()
num = 0
for i in toy_data_dict.keys():
    L[num] = 0
    if num in complete_jobs:
        tasks_new.append(toy_data_dict[i]['stage']['2'])
        L[num] = len(toy_data_dict[i]['stage']['2'])
    else:
        tasks_new.append([])
    num += 1
M = 0
for i in K:
    M += J * L[i]

# 多轮调度接口实现

In [656]:
def release_resource():
    complete_time_order = sorted(complete_time.items(), key=lambda x:x[1])
    num = 0
    wait = 0
    start = complete_time_order[0][1]
    remained_jobs = w - len(job_complete_time)
    complete_jobs.clear()
    for i in range(min(job_threshold,remained_jobs)):
        wait = (complete_time_order[i][1] - start)
        if(wait > time_threshold):
            global_time = start + time_threshold
            break
        global_time = complete_time_order[i][1]
        complete_jobs.append(complete_time_order[i][0])
    num = 0
    for i in complete_jobs:
        if(stages[i] == total_stages[i]):
            complete_jobs.remove(i)
            job_complete_time[i] = complete_time_order[num][1]
            del complete_time[i]
        stages[i] += 1
        for j in occ_slots[i]:
            avai[j] += 1
        occ_slots[i].clear()
        num += 1
    return global_time, complete_jobs, stages, job_complete_time, avai, occ_slots

In [657]:
def set_next_iteration():
    tasks_new.clear()
    K = complete_jobs.copy()
    num = 0
    for i in toy_data_dict.keys():
        L[num] = 0
        if num in complete_jobs:
            tasks_new.append(toy_data_dict[i]['stage'][str(stages[num])])
            L[num] = len(toy_data_dict[i]['stage'][str(stages[num])])
        else:
            tasks_new.append([])
        num += 1
    M = 0
    for i in K:
        M += J * L[i]
    e.clear()
    num = 0
    for i in toy_data_dict.keys():
        exc = []
        for j in toy_data_dict[i]['Execution Time'].keys():
            if j in tasks_new[num]:
                exc.append(toy_data_dict[i]['Execution Time'][j])
        e.append(exc)
        num += 1
    c.clear()
    for k in range(w):
        com = []
        for i in range(len(tasks_new[k])):
            compute = []
            for j in range(J):
                maximum = 0
                for loc in demand[tasks_new[k][i]]:
                    src = location[loc] - 1
                    time = demand[tasks_new[k][i]][loc]/bandwidth[src][j]
                    if(time > maximum):
                        maximum = time
                compute.append(maximum)
            com.append(compute)
        c.append(com)
        
    return K, tasks_new, L, M, e, c

# 例：用接口实现全过程调度

In [658]:
while(len(complete_time) != 0):
    global_time, complete_jobs, stages, job_complete_time, avai, occ_slots = release_resource()
    K, tasks_new, L, M, e, c = set_next_iteration()
    for i in range(len(complete_jobs)):
        res = LP(K,L,avai,M)
        K, avai, M = update(K, avai, M)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 94 rows, 117 columns and 234 nonzeros
Model fingerprint: 0xa4b5dad4
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 8e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 94 rows and 117 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6890417e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  1.689041746e+04
16890.417460381326
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 39 columns and 78 nonzeros
Model fingerprint: 0x089df074
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range 

  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 40 rows and 39 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.4918014e+03   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  4.491801449e+03
4491.8014490249125
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 40 rows, 39 columns and 78 nonzeros
Model fingerprint: 0x4d4f92fe
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+07]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 40 rows and 39 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Tim

In [659]:
job_complete_time

{0: 2.0,
 1: 5.511111111111111,
 2: 7.622222222222222,
 3: 8.177777777777777,
 4: 19.555555555555557,
 5: 24.796825396825398}